<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/352_WDO_Prioritization_Node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Workforce Development Orchestrator — Prioritization Node

The `prioritization_node` is the **decision convergence point** of the Workforce Development Orchestrator. Its job is to take *many valid insights* and turn them into a **small, ordered set of actions leaders can actually take**.

This node does not invent priorities.
It **enforces them** — using transparent, configurable policy.

---

## 1. This Node Is Where Strategy Becomes Choice

Upstream nodes answer important questions:

* Where is automation risk?
* Which skills are missing?
* What learning paths fit?
* Which roles may need to evolve?

But leaders don’t act on *lists* — they act on **ranked trade-offs**.

This node answers:

> **“What should we do first?”**

And it does so deterministically.

---

## 2. All Major Recommendation Types Are Reconciled Together

The node brings together four competing domains:

* Skill gaps (human capability risk)
* Learning paths (investment options)
* Role evolutions (structural change)
* Workforce-level metrics (strategic health)

Handling these in one place ensures that prioritization is:

* Comparable
* Consistent
* Governable

No single signal dominates by accident.

---

## 3. Policy-Driven Prioritization (Not Heuristics)

Each prioritization step delegates to **toolshed prioritization utilities**, which apply:

* Normalized scoring
* Explicit weights
* Deterministic sorting

Crucially:

* The node does not define *how* to prioritize
* It defines *when* and *what* to prioritize

That keeps orchestration clean and policy centralized.

---

## 4. Top-N Limits Enforce Focus

The use of configurable limits:

* `top_n_gaps`
* `top_n_learning_paths`
* `top_n_role_evolutions`

is not cosmetic — it’s strategic.

This ensures:

* Leaders are not overwhelmed
* Execution bandwidth is respected
* The system encourages sequencing, not panic

Changing these values instantly adjusts **decision density** without touching logic.

---

## 5. Workforce Summary Anchors Decisions in Reality

The `calculate_workforce_summary` call grounds prioritization in **organizational context**, answering questions like:

* How many employees are at risk?
* How many gaps are high priority?
* Are learning recommendations covering the workforce?
* How ready is the organization overall?

This prevents a classic failure mode:

> Acting on individual recommendations without understanding systemic impact.

The summary keeps leaders oriented at the **portfolio level**.

---

## 6. Deterministic, Auditable, Repeatable

This node has no randomness, no model calls, no hidden inference.

Given the same inputs and config:

* It will always produce the same output
* Changes in outcome can be traced to:

  * Data changes, or
  * Policy changes

That makes it:

* Auditable
* Testable
* Defensible in executive and board settings

---

## 7. Errors Are Preserved, Not Suppressed

As with the rest of the agent, unexpected issues are:

* Captured
* Appended
* Returned visibly

This ensures the system never silently fails while still allowing partial insight delivery.

---

## Why This Node Is the “Trust Moment”

From a leadership perspective, this node guarantees:

* Priorities reflect stated policy
* Trade-offs are explicit
* Focus is enforced
* Outcomes are reproducible

From a systems perspective, it guarantees:

* Clear separation of orchestration vs logic
* Easy extensibility
* Zero reliance on stochastic behavior

This is **decision infrastructure**, not AI suggestion.

---

## Architectural Takeaway

This node embodies the central philosophy of your agent:

> **AI should not decide what matters — leadership should.
> The system should enforce that decision consistently.**

By centralizing prioritization, applying explicit policy, and enforcing focus, the Workforce Development Orchestrator earns the right to influence real organizational action.




In [ ]:
def prioritization_node(
    state: WorkforceDevelopmentOrchestratorState,
    config: WorkforceDevelopmentOrchestratorConfig
) -> Dict[str, Any]:
    """
    Prioritization Node: Orchestrate prioritizing gaps, learning paths, and role evolutions.

    Uses toolshed prioritization utilities to prioritize all recommendations.
    """
    errors = state.get("errors", [])

    # Get required data from state
    skill_gap_analysis = state.get("skill_gap_analysis", [])
    learning_path_recommendations = state.get("learning_path_recommendations", [])
    role_evolution_recommendations = state.get("role_evolution_recommendations", [])
    employees = state.get("employees", [])
    roles = state.get("roles", [])
    tasks = state.get("tasks", [])
    automation_risk_analysis = state.get("automation_risk_analysis", [])

    try:
        # Prioritize skill gaps
        prioritized_gaps = prioritize_skill_gaps(skill_gap_analysis, config)

        # Prioritize learning paths
        prioritized_learning_paths = prioritize_learning_paths(
            learning_path_recommendations,
            config
        )

        # Prioritize role evolutions
        prioritized_evolutions = prioritize_role_evolutions(
            role_evolution_recommendations,
            config
        )

        # Calculate workforce summary
        workforce_summary = calculate_workforce_summary(
            employees,
            roles,
            tasks,
            skill_gap_analysis,
            learning_path_recommendations,
            role_evolution_recommendations,
            automation_risk_analysis
        )

        return {
            "prioritized_gaps": prioritized_gaps[:config.top_n_gaps],
            "prioritized_recommendations": prioritized_learning_paths[:config.top_n_learning_paths],
            "prioritized_evolutions": prioritized_evolutions[:config.top_n_role_evolutions],
            "workforce_summary": workforce_summary,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"prioritization_node: Unexpected error: {str(e)}"]
        }

# Test prioritization node

In [ ]:
"""Test prioritization node

Testing Phase 7: Prioritization Node
Following the pattern: Test node after utilities pass
"""

from agents.workforce_development_orchestrator.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    automation_risk_analysis_node,
    skill_gap_detection_node,
    learning_path_matching_node,
    role_evolution_analysis_node,
    prioritization_node
)
from config import (
    WorkforceDevelopmentOrchestratorState,
    WorkforceDevelopmentOrchestratorConfig
)


def test_prioritization_node():
    """Test prioritization node"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # Load data and run all previous nodes
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    risk_update = automation_risk_analysis_node(state, config)
    state.update(risk_update)

    gap_update = skill_gap_detection_node(state, config)
    state.update(gap_update)

    path_update = learning_path_matching_node(state, config)
    state.update(path_update)

    evolution_update = role_evolution_analysis_node(state, config)
    state.update(evolution_update)

    # Then prioritize
    result = prioritization_node(state, config)

    # Check that prioritized items are present
    assert "prioritized_gaps" in result
    assert "prioritized_recommendations" in result
    assert "prioritized_evolutions" in result
    assert "workforce_summary" in result

    # Check that items are sorted (first should have higher score)
    if len(result["prioritized_gaps"]) > 1:
        assert result["prioritized_gaps"][0]["priority_score"] >= result["prioritized_gaps"][1]["priority_score"]

    # Check summary
    summary = result["workforce_summary"]
    assert "total_employees" in summary
    assert "overall_workforce_readiness_score" in summary

    # Verify no errors
    assert len(result.get("errors", [])) == 0

    print("✅ test_prioritization_node: PASSED")


def test_prioritization_node_top_n():
    """Test prioritization node respects top_n limits"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()
    config.top_n_gaps = 5
    config.top_n_learning_paths = 3
    config.top_n_role_evolutions = 2

    # Load data and run all previous nodes
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    risk_update = automation_risk_analysis_node(state, config)
    state.update(risk_update)

    gap_update = skill_gap_detection_node(state, config)
    state.update(gap_update)

    path_update = learning_path_matching_node(state, config)
    state.update(path_update)

    evolution_update = role_evolution_analysis_node(state, config)
    state.update(evolution_update)

    # Then prioritize
    result = prioritization_node(state, config)

    # Check top_n limits
    assert len(result["prioritized_gaps"]) <= config.top_n_gaps
    assert len(result["prioritized_recommendations"]) <= config.top_n_learning_paths
    assert len(result["prioritized_evolutions"]) <= config.top_n_role_evolutions

    print("✅ test_prioritization_node_top_n: PASSED")


def test_prioritization_integration():
    """Test prioritization with full workflow"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # Full workflow up to prioritization
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    risk_update = automation_risk_analysis_node(state, config)
    state.update(risk_update)

    gap_update = skill_gap_detection_node(state, config)
    state.update(gap_update)

    path_update = learning_path_matching_node(state, config)
    state.update(path_update)

    evolution_update = role_evolution_analysis_node(state, config)
    state.update(evolution_update)

    priority_update = prioritization_node(state, config)
    state.update(priority_update)

    # Verify all prioritized data is present
    assert "prioritized_gaps" in state
    assert "prioritized_recommendations" in state
    assert "prioritized_evolutions" in state
    assert "workforce_summary" in state

    # Verify summary quality
    summary = state["workforce_summary"]
    assert summary["total_employees"] == 10
    assert summary["total_roles"] == 5
    assert 0.0 <= summary["overall_workforce_readiness_score"] <= 100.0

    print("✅ test_prioritization_integration: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Prioritization Node (Phase 7)")
    print("=" * 60)
    print()

    test_prioritization_node()
    test_prioritization_node_top_n()
    test_prioritization_integration()

    print()
    print("=" * 60)
    print("✅ All prioritization node tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_prioritization_node.py
============================================================
Testing Prioritization Node (Phase 7)
============================================================

✅ test_prioritization_node: PASSED
✅ test_prioritization_node_top_n: PASSED
✅ test_prioritization_integration: PASSED

============================================================
✅ All prioritization node tests passed!
============================================================
